# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [8]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
import pandas as pd
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import joblib
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig
from azureml.core import Model

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

I'm using the bank marketing dataset which was provided for previous projects. My code will perform a classification task.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:
data = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")
data_pd = ds.to_pandas_dataframe().dropna()
data_pd

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,...,7,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1,no
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,...,4,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no


In [10]:
 train,test=train_test_split(data_pd,test_size=0.2,random_state=60)

In [9]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'AutoML_experiment'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

I haven't used any special settings. My configuarion includes the experiment timeout to prevent the experiment to take to much time. Also I clarified that this is a classification where the primary metric is accuracy.

In [11]:
# TODO: Put your automl settings here
automl_settings = {}

# TODO: Put your automl config here

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data= train,
    label_column_name='y',
    n_cross_validations= 5)

In [12]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config,show_output=True)

No run_configuration provided, running on local with default configuration


2022-05-15:12:40:55,351 INFO     [modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-05-15:12:40:55,374 INFO     [modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-05-15:12:41:28,126 INFO     [font_manager.py:1337] generated new fontManager


Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML_experiment,AutoML_baa48f77-fb5d-4aec-8819-131a07270c28,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2022-05-15:13:15:17,483 INFO     [explanation_client.py:334] Using default datastore for uploads


Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
********************************************************************************************


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
   RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [55]:
# Retrieve and save your best automl model.

best_run,fitted_model=remote_run.get_output()


In [56]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
AutoML_experiment,AutoML_baa48f77-fb5d-4aec-8819-131a07270c28_35,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [59]:
#TODO: Save the best model
joblib.dump(fitted_model,'automl_best_model')

['automl_best_model']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [60]:
automl_model=remote_run.register_model(model_name='automl_best_model')#,
#                                        model_path='outputs/automml_best_model.joblib')

In [37]:
#automl_model_register=Model.register(ws,model_name='automl_best_model.joblib',model_path='./automl_best_model.joblib')

Registering model automl_best_model.joblib


In [61]:
automl_model

Model(workspace=Workspace.create(name='quick-starts-ws-195691', subscription_id='3e42d11f-d64d-4173-af9b-12ecaa1030b3', resource_group='aml-quickstarts-195691'), name=automl_best_model, id=automl_best_model:1, version=1, tags={}, properties={})

In [67]:
from azureml.core.model import InferenceConfig

environment = best_run.get_context().get_environment()
inference_config = InferenceConfig(entry_script='score.py', environment=environment)

UserErrorException: UserErrorException:
	Message: There is no run definition or environment specified for the run.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "There is no run definition or environment specified for the run."
    }
}

In [69]:
from azureml.core import Environment
from azureml.core.webservice import Webservice

service=Model.deploy(ws, 'automl-webservice',[automl_model])
service.wait_for_deployment(show_output=True)

UserErrorException: UserErrorException:
	Message: You must provide an InferenceConfig when deploying a model with model_framework set to Custom. Default environments are only provided for these frameworks: ['Onnx', 'ScikitLearn', 'TensorFlow', 'Multi'].
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You must provide an InferenceConfig when deploying a model with model_framework set to Custom. Default environments are only provided for these frameworks: ['Onnx', 'ScikitLearn', 'TensorFlow', 'Multi']."
    }
}

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
